가장 처음 풀이.
플로이드-워셜로 풀었는데, N <= 1,000이라 타임아웃인 것 같다.

In [16]:
n_cities, n_roads, party = map(int, input().split())

graph = [[float('inf')] * (n_cities+1) for _ in range(n_cities+1)]
for i in range(1, n_cities+1):
    graph[i][i] = 0

for _ in range(n_roads):
    start, end, dist = map(int, input().split())
    graph[start][end] = dist
    
for start in range(1, n_cities+1):
    for end in range(1, n_cities+1):
        for bypass in range(1, n_cities+1):
            graph[start][end] = min(graph[start][end], graph[start][bypass] + graph[bypass][end])

the_farthest_dist = 0
for start in range(1, n_cities+1):
    the_farthest_dist = max(the_farthest_dist, graph[start][party] + graph[party][start])

print(the_farthest_dist)
    

10


In [ ]:
n_cities, n_roads, party = map(int, input().split())

graph = [[float('inf')] * (n_cities+1) for _ in range(n_cities+1)]
# for i in range(1, n_cities+1):
#     graph[i][i] = 0

for _ in range(n_roads):
    start, end, dist = map(int, input().split())
    graph[start][end] = dist
    
for start in range(1, n_cities+1):
    for end in range(1, n_cities+1):
        for bypass in range(1, n_cities+1):
            if start == end:
                graph[start][end] = 0
            else:
                graph[start][end] = min(graph[start][end], 
                                        graph[start][bypass] + graph[bypass][end])

the_farthest_dist = 0
for start in range(1, n_cities+1):
    the_farthest_dist = max(the_farthest_dist, 
                            graph[start][party] + graph[party][start])

print(the_farthest_dist)
    

다익스트라로 다시 풀어보자
- 다익스트라를 매 도시들마다 반복하는 식으로 풀었다.
    - 그럼 시간복잡도는 $O(VE \log V)$?
    - 얼핏봐도 플로이드보다는 더 빨라보이고, 문제도 풀린다.
    - 그럼 **플로이드-워셜은 언제 쓰는게 좋은가?**



In [10]:
n_cities, n_roads, party = map(int, input().split())

dists = [[float('inf')] * (n_cities + 1) for _ in range(n_cities+1)]
graph = [[] for _ in range(n_cities+1)]

for _ in range(n_roads):
    start, end, dist = map(int, input().split())
    graph[start].append((end, dist))
    
    
import heapq

def dijkstra(start):
    q = []
    dists[start][start] = 0
    heapq.heappush(q, (0, start))
    
    while q:
        print(q)
        current_dist, neareast_node = heapq.heappop(q)
        
        if dists[start][neareast_node] < current_dist:
            continue
        
        for end, dist in graph[neareast_node]:
            detouring_dist = current_dist + dist
            if detouring_dist < dists[start][end]:
                dists[start][end] = detouring_dist
                heapq.heappush(q, (detouring_dist, end))
                
for start_city in range(1, n_cities+1):
    dijkstra(start_city)
    
the_farthest_dist = 0
for start in range(1, n_cities+1):
    the_farthest_dist = max(the_farthest_dist, dists[start][party] + dists[party][start])

print(the_farthest_dist)

[(0, 1)]
[(2, 3), (4, 2), (7, 4)]
[(4, 2), (7, 4), (6, 4)]
[(6, 4), (7, 4)]
[(7, 4)]
[(0, 2)]
[(1, 1), (5, 3)]
[(3, 3), (5, 3), (8, 4)]
[(5, 3), (8, 4), (7, 4)]
[(7, 4), (8, 4)]
[(8, 4)]
[(0, 3)]
[(2, 1), (4, 4)]
[(4, 4), (6, 2)]
[(6, 2)]
[(0, 4)]
[(3, 2)]
[(4, 1), (8, 3)]
[(6, 3), (8, 3)]
[(8, 3)]
10


- 더 효율적인 풀이 로직과 참고 코드
    - https://www.acmicpc.net/board/view/14990



In [12]:
## https://www.acmicpc.net/source/64071827
# import sys; input = sys.stdin.readline
from math import inf
from heapq import heappop, heappush

def dijkstra(start, dist, idx):
    pq = [(0, start)]
    dist[start] = 0
    while pq:
        print(pq)
        t, s = heappop(pq)
        if dist[s] < t:
            continue
        for e, tt in graph[idx][s]:
            if dist[e] > t + tt:
                dist[e] = t + tt
                heappush(pq, (dist[e], e))

N, M, X = map(int, input().split())
# X -= 1

graph = [[[] for _ in range(N+1)] for _ in range(2)]
for _ in range(M):
    S, E, T = map(int, input().split())
    # S -= 1; E -= 1
    graph[0][S].append((E, T))  ## 목적지 -> 각 노드
    graph[1][E].append((S, T))  ## 그래프를 뒤로 따라가면, 그게 곧 각 노드 -> 목적지

dist = [[inf] * (N+1) for _ in range(2)]
dijkstra(X, dist[0], 0)
dijkstra(X, dist[1], 1)

print(max(dist[0][i] + dist[1][i] for i in range(1, N+1)))

[(0, 2)]
[(1, 1), (5, 3)]
[(3, 3), (5, 3), (8, 4)]
[(5, 3), (8, 4), (7, 4)]
[(7, 4), (8, 4)]
[(8, 4)]
[(0, 2)]
[(3, 4), (4, 1)]
[(4, 1), (7, 3)]
[(6, 3), (7, 3)]
[(7, 3)]
10


- 플로이드 + pypy3 조합으로 풀린 코드
    - 플로이드 알고리즘에 대한 약간의 최적화?


출처: https://claude-u.tistory.com/338

In [ ]:
import sys

#입력
N, M, X = map(int, input().split())
distance = [[M * 100 for _ in range(N+1)] for _ in range(N+1)]
for _ in range(M):
    start, end, time = map(int, sys.stdin.readline().split())
    distance[start][end] = time


#플로이드 와샬 알고리즘
for k in range(1, N+1): #경로 for문이 가장 상위 단계여야 누락되지 않는다
    for i in range(1, N+1):
        if distance[i][k] != M * 100:
            for j in range(1, N+1):
                if i == j: #자기 자신일 경우
                    distance[i][j] = 0 
                else:
                    distance[i][j] = min(distance[i][j],
                                         distance[i][k] + distance[k][j])


#출력
max_time = 0
for i in range(1, N+1):
    max_time = max(max_time,
                   distance[i][X] + distance[X][i])
print(max_time)